In [19]:
%load_ext autoreload
%autoreload

import os
import sys
import django
from IPython.core.debugger import set_trace
from pathlib import Path
from django.core.files.base import ContentFile
from rest_framework import serializers

parentdir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parentdir)
django.setup()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import getch.models as m

In [3]:
#m.MaskBase.objects.all().delete()

In [4]:
m.MaskBase.objects.all();

# MaskBase

In [119]:
dir_masks = Path(parentdir) / 'getch/static/materials/masks'

In [120]:
for d in dir_masks.glob('*'):
    if d.name != 'archives':
        _type = d.name.upper()
        
        for f in d.rglob('*.*'):
            cat = f.parts[-2]
            
            if cat != 'raw':
                maskbase = m.MaskBase.objects.create(type=_type, category=cat)
                maskbase.pix.save(f.name, ContentFile(f.read_bytes()), save=True)

# MaskSerializer

In [38]:
class MaskSerializer(serializers.ModelSerializer):
    class Meta:
        model = m.Mask
        fields = ['id', 'maskbase', 'top', 'left', 'width', 'height']
        read_only_fields = ['id']
        
MaskSerializer()

MaskSerializer():
    id = IntegerField(read_only=True)
    maskbase = PrimaryKeyRelatedField(queryset=MaskBase.objects.all())
    top = FloatField(required=False)
    left = FloatField(required=False)
    width = FloatField(required=False)
    height = FloatField(required=False)

## Serializing

In [21]:
mask = m.Mask.objects.all()[0]
data = MaskSerializer(mask).data; data

{'id': 2, 'maskbase': 14, 'top': 0.0, 'left': 0.0, 'width': 100.0, 'height': 20.0}

## Deserializing
### 1. Create

In [24]:
serializer = MaskSerializer(data={'maskbase':15, 'top':10, 'left':0, 'width':50, 'height':50})
serializer.is_valid()
mask = serializer.save(); mask

<Mask: EYE | animal | 2020-08-07_110328.890580SIDEB_EYEMASK_A_ANIMAL_02_cat.jpg | T=10.0, L=0.0, W=50.0, H=50.0>

### 2. Update

In [25]:
serializer = MaskSerializer(mask, data={'maskbase':16, 'top':0, 'left':0, 'width':50, 'height':50})
serializer.is_valid()
mask = serializer.save(); mask

<Mask: EYE | animal | 2020-08-07_110330.219042SIDEB_EYEMASK_A_ANIMAL_02_dog.jpg | T=0.0, L=0.0, W=50.0, H=50.0>

# ProfileSerializer

In [50]:
class ProfileSerializer(serializers.ModelSerializer):
    eye_mask = MaskSerializer(required=False)
    mouth_mask = MaskSerializer(required=False)

    class Meta:
        model = m.Profile
        fields = ['id', 'type', 'pix', 'image', 'text', 'eye_mask', 'mouth_mask']
        read_only_fields = ['id']
        
ProfileSerializer()

ProfileSerializer():
    id = IntegerField(read_only=True)
    type = ChoiceField(choices=(('IMAGE', 'image'), ('TEXT', 'text')), required=False)
    pix = ImageField(allow_null=True, max_length=500, required=False)
    image = ImageField(allow_null=True, max_length=500, required=False)
    text = CharField(allow_blank=True, allow_null=True, max_length=10, required=False)
    eye_mask = MaskSerializer(required=False):
        id = IntegerField(read_only=True)
        maskbase = PrimaryKeyRelatedField(queryset=MaskBase.objects.all())
        top = FloatField(required=False)
        left = FloatField(required=False)
        width = FloatField(required=False)
        height = FloatField(required=False)
    mouth_mask = MaskSerializer(required=False):
        id = IntegerField(read_only=True)
        maskbase = PrimaryKeyRelatedField(queryset=MaskBase.objects.all())
        top = FloatField(required=False)
        left = FloatField(required=False)
        width = FloatField(required=False)

## Serializing

In [51]:
profile = m.Profile.objects.all()[0]
data = ProfileSerializer(profile).data; data

{'id': 7, 'type': 'IMAGE', 'pix': 'https://storage.googleapis.com/sideb-proejct.appspot.com/user/sejin76%40nate.com/2020-08-05_093216.331442__mypix.png?Expires=1596880342&GoogleAccessId=sideb-serviceaccount%40sideb-proejct.iam.gserviceaccount.com&Signature=RiHZAVVdIj54j0bByXA0pRc2mkuGZigbLj3lZnd6Cl1jHc85I1dfcGUEwYxCsml%2FqvMeOZspLv7Ooc86zaZ80UcDDwpB3hhpSAb9KOsJx0Zj7O0xeBxuVTpfLi09wh2ruFr15JqdyfcB%2FjKM49Ba52kZZgaft5hNK1Wz1tTTAbkYRWIF333wsfiGJRIfzQj6ldz4hT54dVxRRXocKcgpGab3qZPM4FfeJKVw7SNVrUzFFhWnYOLBJXWsQUwXSMdKUMVL8SMbG5utE6q%2BQKUuaUsJeq05xGBGRTi%2F0GBMvVo8K%2B9PM8p0FPhGlf4u7qujNbR0KlOCFDRkKuM%2FAwLZKA%3D%3D', 'image': None, 'text': None, 'eye_mask': None, 'mouth_mask': None}

## Deserializing
### 1. create

In [52]:
pix = m.PostVoteOX.objects.all()[0].pix

In [55]:
serializer = ProfileSerializer(data={'type':'IMAGE', 'text':'TESTTEST', 'pix':pix, 'image':None, 'eye_mask':None, 'mouth_mask':None})
serializer.is_valid()
profile = serializer.save(); profile

AssertionError: You cannot call `.save()` on a serializer with invalid data.

In [46]:
m.PostVoteOX.objects.all()[0].pix

<ImageFieldFile: post/2020/7/22/sejin76@nate.com/2020-07-22_195937.085170__AB_anorak_B.jpg>